# 线性回归模型，自定义梯度

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import sklearn
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras
print(tf.__version__)
print(sys.version_info)
for module in np, pd ,sklearn, tf, keras:
    print(module.__name__,module.__version__)

2.0.0
sys.version_info(major=3, minor=7, micro=6, releaselevel='final', serial=0)
numpy 1.18.1
pandas 1.0.1
sklearn 0.22.2
tensorflow 2.0.0
tensorflow_core.keras 2.2.4-tf


In [2]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()

print(housing.DESCR)
print(housing.data.shape)
print(housing.target.shape)


.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block
        - HouseAge      median house age in block
        - AveRooms      average number of rooms
        - AveBedrms     average number of bedrooms
        - Population    block population
        - AveOccup      average house occupancy
        - Latitude      house block latitude
        - Longitude     house block longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
http://lib.stat.cmu.edu/datasets/

The target variable is the median house value for California districts.

This dataset was derived from the 1990 U.S. census, using one row per census
block group. A block group is the smallest geographical unit for which the U.S.
Census Bur

In [3]:
from sklearn.model_selection import train_test_split

x_train_all,x_test,y_train_all,y_test=train_test_split(housing.data, housing.target,random_state=7)
x_train, x_valid, y_train, y_valid = train_test_split(x_train_all,y_train_all,random_state=11)

In [4]:
from sklearn.preprocessing import StandardScaler
transfer = StandardScaler()
x_train_scaled = transfer.fit_transform(x_train)
x_test_scaled = transfer.transform(x_test)
x_valid_scaled = transfer.transform(x_valid)

In [5]:
#metric的使用, 均方差
metric = keras.metrics.MeanSquaredError()
print(metric([5.],[2.]))
print(metric([0.],[1.]))
#之前结果会累加记录
print(metric.result())
#如果不想累加
metric.reset_states()#清空
metric([1.],[3.])
print(metric.result())

tf.Tensor(9.0, shape=(), dtype=float32)
tf.Tensor(5.0, shape=(), dtype=float32)
tf.Tensor(5.0, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)


In [9]:
# 1、batch的形式遍历数据集 metric
#      1.1自动求导替换为手动改求导
# 2、epich结束，验证集验证 metric

epochs = 15
batch_size =32
steps_per_epoch = len(x_test_scaled) //batch_size
optimizer = keras.optimizers.SGD()
metric = keras.metrics.MeanSquaredError()
def random_batch(x,y, batch_size=32):
    idx = np.random.randint(0,len(x),size = batch_size)
    return x[idx], y[idx] 



model = keras.models.Sequential([
    keras.layers.Dense(30,activation='relu',input_shape=x_train.shape[1:]),
    keras.layers.Dense(1),
    ])

for epoch in range(epochs):
    metric.reset_states()
    for step in range(steps_per_epoch):
        x_batch, y_batch = random_batch(x_train_scaled, y_train, batch_size)
        with tf.GradientTape() as tape:
            y_pred = model(x_batch)
            loss  = tf.reduce_mean(# 降低均值
#                 tf.reduce_mean 函数用于计算张量tensor沿着指定的数轴（tensor的某一维度）上的的平均值，
#                 默认axis=None, keep_dims=False,计算所有元素的均值
                    keras.losses.mean_squared_error(y_batch, y_pred))
            metric(y_batch,y_pred)
        grads = tape.gradient(loss, model.variables)#自动求解每个变量和loss的梯度
        grads_and_vars = zip(grads, model.variables )#将变量与梯度对应，
        optimizer.apply_gradients(grads_and_vars)#使用optimizer.apply_gradients完成梯度下降。
        print('\r Epoch',epoch,'train_loss',loss.numpy(),'train mse:',metric.result().numpy(),end = ' ')
    y_valid_pred = model(x_valid_scaled)#验证，每周期完成一次验证。
    valid_loss  = tf.reduce_mean(
            keras.losses.mean_squared_error(y_valid_pred, y_valid))
    print('/t', 'valid mse: ', valid_loss.numpy())


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

 Epoch 0 train_loss 1.7642136 train mse: 1.6223273  /t valid mse:  1.530690806061189
 Epoch 1 train_loss 3.8490808 train mse: 5.4682555  /t valid mse:  2.956105838713065
 Epoch 2 train_loss 1.2618407 train mse: 1.75555 39 /t valid mse:  1.5646127585395502
 Epoch 3 train_loss 0.7100762 train mse: 1.7005972  /t valid mse:  1.491883889596949
 Epoch 4 train_loss 1.9047118 train mse: 1.3167212  /t valid mse:  1.4087120753929327
 Epoch 5 train_loss 1.4936867 train mse: 1.314371   /t valid mse:  1.403195216041